# EfficientNetB0


## I. Setup


In [78]:
import os
import sys

# os.chdir('/Volumes/SD Card/cleaned')
# os.getcwd()

sys.path.append('..')

In [79]:
dataset_path = '/Volumes/SD Card/cleaned'


In [80]:
def listdir(path: str):
    return os.listdir(path)


cosonant_path = f'{dataset_path}/cosonant'
independent_vowel_path = f'{dataset_path}/independent_vowel'
stack_consonant_vowel_path = f'{dataset_path}/stack_consonant'
sub_vowel_path = f'{dataset_path}/sub_vowel'
vowel_path = f'{dataset_path}/vowel'

In [81]:
count = 0
for inner1 in listdir(f'{dataset_path}'):
    for inner2 in listdir(f'{dataset_path}/{inner1}'):
        try:
            count = count + len(listdir(f'{dataset_path}/{inner1}/{inner2}'))
        except NotADirectoryError as e:
            # print(f'ERROR: {e}')
            continue

# consider my laptop slow, 
# should only run 100 per epoch
epochs = count / 100
epochs

331.68

## II. Prepare dataset


In [82]:
import pandas as pd
# import random


In [83]:
df = pd.DataFrame(columns=["label", "path"])

In [84]:
for cosonant in listdir(cosonant_path):
    variants_path = f'{cosonant_path}/{cosonant}'
    variants = listdir(f'{cosonant_path}/{cosonant}')

    for variant in variants:
    # for variant in random.sample(variants, 4):
    # for variant in [variants[0], variants[1], variants[2], variants[3]]:
        vpath = f'{variants_path}/{variant}'
        df.loc[len(df.index)] = [cosonant, vpath]

In [85]:
df.shape

(132, 2)

In [86]:
df.head(5)

,label,path
0,ba,/Volumes/SD Card/cleaned/cosonant/ba/119.jpg
1,ba,/Volumes/SD Card/cleaned/cosonant/ba/120.jpg
2,ba,/Volumes/SD Card/cleaned/cosonant/ba/121.jpg
3,ba,/Volumes/SD Card/cleaned/cosonant/ba/122.jpg
4,cha,/Volumes/SD Card/cleaned/cosonant/cha/122.jpg


In [87]:
df["label"].unique()


array(['ba', 'cha', 'chha', 'chho', 'cho', 'da', 'do', 'ha', 'ka', 'kha',
       'kho', 'ko', 'la', 'lo', 'mo', 'na', 'ngo', 'nho', 'no', 'or',
       'pha', 'pho', 'po', 'ro', 'sa', 'ta', 'tha1', 'tha2', 'tho1',
       'tho2', 'to', 'vo', 'yo'], dtype=object)

## III. Preprocessing


### a. Image


In [88]:
import tensorflow as tf
import cv2
import numpy as np

images = []
labels = []
error_count = 0


def resize_image(image_path):
    image = cv2.imread(image_path)
    image = tf.image.resize(image, size=(224, 224))
    image = tf.cast(image, dtype=tf.float32)
    return image


for item in np.array(df):
    label = item[0]
    path = item[1]

    try:
        image = resize_image(path)
        images.append(image)
        labels.append(label)
    except ValueError as e:
        error_count = error_count + 1

error_count, len(images), len(labels)


(0, 132, 132)

In [89]:
images = np.array(images)
labels = np.array(labels)


### b. Label


In [90]:
from sklearn.preprocessing import LabelEncoder

labelEncoder = LabelEncoder()
y = labelEncoder.fit_transform(labels)
Y = y.reshape(-1, 1)

Y.shape


(132, 1)

### c. Split testing & training data


In [91]:
from sklearn.model_selection import train_test_split


In [92]:
train_x, test_x, train_Y, test_Y = train_test_split(
    images,
    Y,
    test_size=0.2,
    random_state=10
)


In [93]:
train_x.shape, test_x.shape, train_Y.shape, test_Y.shape


((105, 224, 224, 3), (27, 224, 224, 3), (105, 1), (27, 1))

## IV. Train


In [94]:
NUM_CLASSES = 3
IMG_SIZE = 224

size = (IMG_SIZE, IMG_SIZE)


In [95]:
inputs = tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
outputs = tf.keras.applications.efficientnet.EfficientNetB0(
    include_top=True,
    weights=None,
    classes=NUM_CLASSES
)(inputs)

model = tf.keras.Model(inputs, outputs)
model.compile(
    optimizer="sgd",
    loss="mse",
    metrics=["accuracy"]
)

model.summary()


Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, 3)                4053414   
                                                                 
Total params: 4,053,414
Trainable params: 4,011,391
Non-trainable params: 42,023
_________________________________________________________________


In [96]:
from datetime import datetime

In [97]:
epochs = 4

hist = model.fit(train_x, train_Y, epochs=epochs, verbose=2)
trained_at = datetime.now().time()

hist.history

Epoch 1/4
4/4 - 33s - loss: 353.8961 - accuracy: 0.0476 - 33s/epoch - 8s/step
Epoch 2/4
4/4 - 22s - loss: 353.8979 - accuracy: 0.0476 - 22s/epoch - 6s/step
Epoch 3/4
4/4 - 23s - loss: 353.8838 - accuracy: 0.0286 - 23s/epoch - 6s/step
Epoch 4/4
4/4 - 41s - loss: 353.8795 - accuracy: 0.0381 - 41s/epoch - 10s/step


In [98]:
prediction = model.evaluate(test_x, test_Y)

print("Loss = " + str(prediction[0]))
print("Test Accuracy = " + str(prediction[1]))

1/1 [==============================] - 7s 7s/step - loss: 267.4696 - accuracy: 0.0000e+00
Loss = 267.4695739746094
Test Accuracy = 0.0


## V. Save Model

In [99]:
model.save(f'../models/efficient_net_b0-{trained_at}.h5')